In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

class PSNRMetric(nn.Module):
    def forward(self, y_true, y_pred):
        y_pred = torch.clamp(y_pred, 0, 1)
        mse = torch.mean((y_true - y_pred) ** 2)
        psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
        return psnr.item()

class SSIMMetric(nn.Module):
    def forward(self, y_true, y_pred):
        y_pred = torch.clamp(y_pred, 0, 1)
        ssim = TF.ssim(y_true, y_pred, data_range=1.0)
        return torch.mean(ssim).item()

class Model(nn.Module):
    def __init__(self, filters=64, upscale_factor=2):
        super(Model, self).__init__()
        self.relu = nn.ReLU()
        self.conv_skip1 = nn.Conv2d(1, filters, kernel_size=3, padding='same')
        self.conv_inner = nn.ModuleList([nn.Conv2d(filters, filters, kernel_size=3, padding='same') for _ in range(16)])
        self.conv_skip2 = nn.Conv2d(filters, filters, kernel_size=3, padding='same')
        self.conv_out = nn.Conv2d(filters, 4, kernel_size=3, padding='same')
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self.initialize_weights()

    def forward(self, input):
        x = self.conv_skip1(input)
        x0 = x
        for conv_layer in self.conv_inner:
            x = self.relu(conv_layer(x))
        x1 = self.conv_skip2(x)
        x2 = self.conv_out(x1 + x0)
        output = self.pixel_shuffle(x2)
        return output

    def initialize_weights(self):
        init.xavier_uniform_(tensor=self.conv_skip1.weight)
        for conv_layer in self.conv_inner:
            init.kaiming_uniform_(tensor=conv_layer.weight, nonlinearity='relu')
        init.xavier_uniform_(tensor=self.conv_skip2.weight)
        init.xavier_uniform_(tensor=self.conv_out.weight)

model = Model()
model.load_state_dict(torch.load("/content/model.pth"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/Datasets/DIV2K_Train_LR.zip /content/LR.zip
!cp /content/drive/MyDrive/Datasets/DIV2K_Train_HR.zip /content/HR.zip

!unzip /content/LR.zip
!unzip /content/HR.zip

In [ ]:
!rm -rf /content/HR
!rm -rf /content/LR

In [ ]:
# Load data into memory
import cv2
import numpy as np
import glob

filelist1 = sorted(glob.glob('/content/LR/*'))
train_in = []
for myFile in filelist1:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    train_in.append(image)
train_in = np.array(train_in).astype(np.float32) / 255.0
train_in = np.expand_dims(train_in, axis = 1)

filelist2 = sorted(glob.glob('/content/HR/*'))
train_ref = []
for myFile in filelist2:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    train_ref.append(image)
train_ref = np.array(train_ref).astype(np.float32) / 255.0
train_ref = np.expand_dims(train_ref, axis = 1)

# Convert data to PyTorch tensors
train_in_tensor = torch.tensor(train_in)
train_ref_tensor = torch.tensor(train_ref)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms.functional as TF

# Create DataLoader for training data
train_dataset = TensorDataset(train_in_tensor, train_ref_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Move model to device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    # Calculate average loss for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)

    # Print epoch statistics
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.8f}')

print('Finished Training')
torch.save(model.state_dict(), "/content/model.pth")

In [ ]:
import torch
import cv2
import numpy as np

image = cv2.imread('/content/downscaled.png', cv2.IMREAD_GRAYSCALE)
image = np.array(image).astype(np.float32) / 255.0
image = np.expand_dims(image, axis=0)
image = torch.tensor(image)

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image = image.to(device)

with torch.no_grad():
    output = model(image)

output = output.cpu().numpy()
output = np.squeeze(output)
output = np.clip(output, 0.0, 1.0)
output = np.around(output * 255.0)
output = output.astype(np.uint8)

cv2.imwrite('/content/prediction.png', output)

In [ ]:
!pip install onnx
import onnx

x = torch.ones((1, 1, 1920, 1080))  # N x C x W x H
x = x.to(device)
torch.onnx.export(
    model, x, '/content/c16f64.onnx',
    input_names = ['input'],
    output_names = ['output'],
    dynamic_axes={
        'input' : {0 : 'batch', 2: 'width', 3: 'height'},
        'output' : {0 : 'batch', 2: 'width', 3: 'height'},
    }
)